In [4]:
from fileoperations.fileoperations import getFileNamesInDir
from symbtrdataextractor import symbtrreader, extractor
import os
import json
import pandas as pd 
import numbers

import pdb


In [5]:
usuldict = json.load(open('./unittests/data/usul_extended.json','r'))

symbTrfolder = '../'
symbTrTxtfolder = os.path.join(symbTrfolder, 'txt')
symbTrMu2Folder = os.path.join(symbTrfolder, 'mu2')

symbTr_work_file = os.path.join(symbTrfolder, 'symbTr_mbid.json')

txtfiles, dummy, symbtrnames = getFileNamesInDir(symbTrTxtfolder, keyword = '*.txt')
symbtrnames = [os.path.splitext(s)[0] for s in symbtrnames]

mu2files = [os.path.join(symbTrMu2Folder, sn + '.mu2') for sn in symbtrnames]

symbTrTxtSavefolder = os.path.join('..', '..', 'symbtr_corrections', 'txt')
if not os.path.exists(symbTrTxtSavefolder):
    os.makedirs(symbTrTxtSavefolder)


In [6]:
symbTr_work = json.load(open(symbTr_work_file, 'r'))

for i, (tf, mf, sn) in enumerate(zip(txtfiles, mu2files, symbtrnames)):
    saveTxtfile = os.path.join(symbTrTxtSavefolder, sn + '.txt')
    if not sn[0:2] == '._':
        print str(i) + ' ' + sn
        try:
            mu2header = symbtrreader.readMu2Header(mf)[0] 

            for usul in usuldict.values():
                for uv in usul['variants']:
                    if uv['mu2_name'] == mu2header['usul']['mu2_name']:
                        mertebe = uv['mertebe']
                        zaman = uv['num_pulses']
                        break

            df = pd.read_csv(tf, sep='\t')
            for index, row in df.iterrows():
                # recompute the erroneous gracenotes with non-zero duration
                if row['Kod'] == 8 and row['Ms'] > 0:
                    row['Pay'] = 0
                    row['Payda'] = 0
                    row['Ms'] = 0

                # recompute zaman and mertebe, if we hit kod 51
                if row['Kod'] == 51:
                    zaman = row['Pay']
                    mertebe = row['Payda']
                    offset_incr = 0
                else:
                    # compute offset
                    offset_incr = 0 if row['Payda'] ==0 else float(row['Pay'])/row['Payda']*mertebe/zaman
                if index == 0:
                    row['Offset'] = offset_incr
                else:
                    prev_row = df.iloc[index-1]
                    row['Offset'] = offset_incr + prev_row['Offset']

                # change null to empty string
                for key, val in row.iteritems():
                    if pd.isnull(val):
                        row[key] = ''

                # make sure that "Sira" column continues consecutively
                row['Sira'] = index+1

                # reassign
                df.iloc[index] = row
            df.to_csv(saveTxtfile, sep='\t', index=False)

            txtdata = extractor.extract(saveTxtfile)[0]
            if not (round(row['Offset']*10000)*0.0001).is_integer():
                print "Ends prematurely!"
        except:
            print "Unexpected error"

0 acem--ilahi--duyek--aldanma_dunya--zekai_dede
1 acem--ilahi--nimevsat--calabim_bir--haci_bayram_veli
2 acem--kupe--duyek--zulfunu--ahmet_avni_konuk
3 acem--selam--devrikebir--asik-i_ger--huseyin_fahreddin_dede
    179.0: men   and Ez  are in the same measure!
    171.0: men   and Fey  are in the same measure!
    163.0: men   and Ez  are in the same measure!
    155.0: men   and Yâ  are in the same measure!
    150.0: dost   and Pîr_ are in the same measure!
    143.0: red   and Ah  are in the same measure!
    140.0: rîz   and Her  are in the same measure!
    137.0: red   and Ez  are in the same measure!
    134.0: red   and Bâ are in the same measure!
    76.0: tim   and Ah  are in the same measure!
    3.0: men   and Ah  are in the same measure!
acem--selam--devrikebir--asik-i_ger--huseyin_fahreddin_dede, 106, VOCAL_SECTION does not start on a measure: 3.85714285714
acem--selam--devrikebir--asik-i_ger--huseyin_fahreddin_dede, 627, VOCAL_SECTION does not start on a measure: 76.5
a